# Part 1: Player Speed & Distance Extraction (2015-2016 Season)

**Games:** 1-600 (out of 1230)

**Output:** `pt1_movement_2015-2016.csv`

This notebook fetches player tracking data from the NBA API's `BoxScorePlayerTrackV3` endpoint for the first 600 games of the 2015-16 season.

In [ ]:
from nba_api.stats.endpoints import (
    leaguegamefinder,
    BoxScorePlayerTrackV3
)
import pandas as pd
from time import sleep

## Step 1: Get game IDs 1-600

In [ ]:
# Grab all games from 2015-16 season
gamefinder = leaguegamefinder.LeagueGameFinder(
    season_nullable='2015-16',
    league_id_nullable='00',
    season_type_nullable='Regular Season'
)

games = gamefinder.get_data_frames()[0]
all_game_ids = games['GAME_ID'].unique().tolist()

# Take first 600 games
game_ids = all_game_ids[0:600]

print(f"Total games in season: {len(all_game_ids)}")
print(f"Processing games 1-600: {len(game_ids)} games")

## Step 2: Define helper function

In [ ]:
def get_game_tracking_data(game_id: str) -> pd.DataFrame | None:
    """Fetch player tracking stats for a single game."""
    try:
        response = BoxScorePlayerTrackV3(
            game_id=game_id,
            timeout=60
        )

        df = response.player_stats.get_data_frame()
        
        if df.empty:
            return None
            
        # Keep only relevant columns
        df = df[['personId', 'firstName', 'familyName', 'speed', 'distance']]
        return df

    except Exception as e:
        print(f"[ERROR] game_id={game_id}: {e}")
        return None

## Step 3: Fetch data (games 1-600)

In [ ]:
all_player_stats = []

for i, game_id in enumerate(game_ids, start=1):
    game_data = get_game_tracking_data(game_id)
    
    if game_data is not None:
        all_player_stats.append(game_data)
    
    # Rate limiting - 2 seconds between requests
    sleep(2)
    
    # Progress update every 50 games
    if i % 50 == 0:
        print(f"[INFO] Processed {i}/600 games")

print(f"\n[COMPLETE] Successfully fetched data for {len(all_player_stats)} games")

## Step 4: Save raw data to CSV

In [ ]:
# Combine all game data
combined_df = pd.concat(all_player_stats, ignore_index=True)

# Create full_name column
combined_df['full_name'] = combined_df['firstName'] + ' ' + combined_df['familyName']

# Save to CSV (raw data, not aggregated yet)
output_path = '../../data/pt1_movement_2015-2016.csv'
combined_df.to_csv(output_path, index=False)

print(f"Saved {len(combined_df)} rows to {output_path}")